<a href="https://colab.research.google.com/github/WoodenPlancks/csc2611/blob/main/CSC2611_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1. Import NLTK in Python: http://www.nltk.org/. Download the Brown Corpus http://www.nltk.org/book/ch02.html for analyses below.

In [ ]:
import nltk
nltk.download('brown')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import brown
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
from string import punctuation
from collections import Counter
import pandas as pd

stopword_set = set(stopwords.words('english'))

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Step 2. Extract the 5000 most common English words (denoted by W) based on unigram frequencies in the Brown corpus. Report the 5 most and least common words you have found in the 5000 words.

In [ ]:
W_lower = [w.lower() for w in brown.words()]
W_no_punc = [w.strip(punctuation) for w in W_lower]
W_nonstop_list = [w for w in W_no_punc if w != ""]
# W_nonstop_list = [w for w in W_no_punc if w not in stopword_set and w != ""]
W_nonstop = Counter(W_nonstop_list)

print("Most common non-stopwords \n", W_nonstop.most_common()[:5])
print("Least common non-stopwords \n", W_nonstop.most_common()[-5:])

W = [wf[0] for wf in W_nonstop.most_common()[:5000]]

Most common non-stopwords 
 [('the', 69971), ('of', 36412), ('and', 28853), ('to', 26158), ('a', 23308)]
Least common non-stopwords 
 [('aviary', 1), ('olive-flushed', 1), ('coral-colored', 1), ('boucle', 1), ('stupefying', 1)]


Update W by adding n words where n is the set of words in Table 1
of RG65 that were not included in the top 5000 words from the Brown corpus. Denote the total number of words in W as |W|.

In [ ]:
# Load in the word pairs and their judged synonymy from Table 1.
table_data = pd.read_csv("Table1.csv")


table_1_list = pd.concat([table_data["Word 1"], table_data["Word 2"]])
table_1_list = list(table_1_list.unique())

for word in table_1_list:
    if word not in W:
        W.append(word)

In [ ]:
W_context = [w for w in W_nonstop_list if w in W]

Step 3. Construct a word-context vector model (denoted by M1) by collecting bigram counts
for words in W. The output should be a |W|×|W| matrix (consider using sparse matrices
for better efficiency), where each row is a word in W, and each column is a context in W
that precedes row words in sentences. For example, if the phrase taxi driver appears 5 times
in the entire corpus, then row taxi and column driver should have a value of 5 in the matrix.

In [ ]:
M1 = pd.DataFrame(0, columns=W, index=W)

# Get bigram counts
bigrams = ngrams(W_nonstop_list, 2)
bigrams_freq = Counter(bigrams)

non_zero_bigrams = Counter({k: c for k, c in bigrams_freq.items() if c > 0})

In [ ]:
for x, c in non_zero_bigrams.items():
    if x[0] in M1.index and x[1] in M1.columns:
        M1.loc[x[0], x[1]] = c

In [ ]:
M1.loc["united", "states"]

396

In [ ]:
total_words = sum([W_nonstop[x] for x in W if x in W_nonstop])
total_words_in_bigrams = M1.sum().sum()

print(total_words, total_words_in_bigrams)

871840 752152


In [ ]:
from math import log2

M1_plus = pd.DataFrame(0, columns=W, index=W)

for x, c in non_zero_bigrams.items():
    if x[0] in M1.index and x[1] in M1.columns:
        pxy = M1.loc[x[0], x[1]] / total_words_in_bigrams
        px = W_nonstop[x[0]]/total_words
        py = W_nonstop[x[1]]/total_words
        # px = sum(M1.loc[x[0]])
        # py = sum(M1[x[1]])
        if pxy > 0:
            # pmi = log2((pxy * total_words_in_bigrams) / (px * py))
            pmi = log2((pxy) / (px * py))
            M1_plus.loc[x[0], x[1]] = max(pmi, 0)

M1_plus

,the,of,and,to,a,in,that,is,was,he,...,voyage,string,wizard,stove,madhouse,jewel,woodland,tumbler,signature,pillow
the,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,...,2.086739,0.604346,3.267311,2.945383,3.852274,3.852274,0.0,0.000000,1.267311,3.174202
of,1.949705,0.000000,0.0,0.0,0.817461,0.00000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
and,0.175391,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.623572,...,0.000000,0.882382,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
to,0.950448,0.000000,0.0,0.0,0.133928,0.00000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
a,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,...,1.350743,3.512206,3.853243,1.531315,0.000000,0.000000,0.0,4.438205,2.853243,2.438205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
jewel,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,6.643390,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
woodland,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
tumbler,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
signature,0.000000,2.209654,0.0,0.0,0.000000,2.98044,0.0,4.058428,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


Step 5. Construct a latent semantic model (denoted by M2) by applying principal components analysis to M1+. The output should return 3 matrices, with different truncated
dimenions at 10 (or a jWj×10 matrix, denoted by M210), 100 (M2100), and 300 (M2300).

In [ ]:
from sklearn.decomposition import PCA

M2_dict = {}
for dim in [10, 100, 300]:
    pca = PCA(n_components = dim)
    M2_dict[dim] = pd.DataFrame(pca.fit_transform(M1_plus))
    M2_dict[dim].index = W

Step 6. Find all pairs of words in Table 1 of RG65 that are also available in W. Denote
these pairs as P. Record the human-judged similarities of these word pairs from the table
and denote similarity values as S.

In [ ]:
table_data.head()

,Word 1,Word 2,Judged Synonymy
0,cord,smile,0.02
1,rooster,voyage,0.04
2,noon,string,0.04
3,fruit,furnace,0.05
4,autograph,shore,0.06


In [ ]:
"cord" in table_data['Word 1'][0]

True

Step 7. Perform the following calculations on each of these models M1, M1+, M210, M2100,
M2300, separately: Calculate cosine similarity between each pair of words in P, based on the
constructed word vectors. Record model-predicted similarities: SM1, SM210, SM2100, SM2300.

In [ ]:
import numpy as np
from numpy.linalg import norm

def cosine_sim(A, B):
    return np.dot(A,B)/(norm(A)*norm(B))
# table_data["Pointw"]

models = {"Word-Context": M1,
          "PPMI": M1_plus,
          "SVD_10": M2_dict[10],
          "SVD_100": M2_dict[100],
          "SVD_300": M2_dict[300]}

for model, matrix in models.items():
    table_data[model] = table_data.apply(lambda row:
                 cosine_sim(matrix.loc[row["Word 1"]],
                            matrix.loc[row["Word 2"]]),
                                         axis = 1)

<ipython-input-13-67b6bf9e712b>:5: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(A,B)/(norm(A)*norm(B))
<ipython-input-13-67b6bf9e712b>:5: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(A,B)/(norm(A)*norm(B))


In [ ]:
table_data.head()

,Word 1,Word 2,Judged Synonymy,Word-Context,PPMI,SVD_10,SVD_100,SVD_300
0,cord,smile,0.02,0.085280,0.001655,-0.464285,-0.198726,-0.121094
1,rooster,voyage,0.04,0.000000,0.000000,0.511314,0.022710,0.029083
2,noon,string,0.04,0.278689,0.014536,0.676875,-0.048881,0.030474
3,fruit,furnace,0.05,0.290191,0.019312,0.162365,-0.027758,0.009038
4,autograph,shore,0.06,0.000000,0.000000,-0.195478,0.011859,-0.031073


Step 8. Report Pearson correlation between S and each of the model-predicted similarities.
Create a GitHub repository that implements all of your analyses; you will need this repo for
the next lab.

In [ ]:
corrs = {}
for model in models.keys():
    corrs[model] = table_data["Judged Synonymy"].corr(table_data[model])
corrs

{'Word-Context': 0.26963380382816016,
 'PPMI': 0.21834604196907115,
 'SVD_10': 0.1345588974345814,
 'SVD_100': 0.35494595528165696,
 'SVD_300': 0.3421407370234144}

# PMI comparison with nltk

In [ ]:
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(W_nonstop_list)
finder.apply_freq_filter(18)
finder.nbest(bigram_measures.pmi, 30)

[('lo', 'shu'),
 ('puerto', 'rico'),
 ('carbon', 'tetrachloride'),
 ('los', 'angeles'),
 ('du', 'pont'),
 ('san', 'francisco'),
 ('electronic', 'switches'),
 ('anode', 'holder'),
 ('sam', 'rayburn'),
 ('prime', 'minister'),
 ('mary', 'jane'),
 ('u', 's'),
 ('rhode', 'island'),
 ('dominant', 'stress'),
 ('brown', 'sharpe'),
 ('nineteenth', 'century'),
 ('miss', 'langford'),
 ('jesus', 'christ'),
 ('roman', 'catholic'),
 ('nuclear', 'weapons'),
 ('armed', 'forces'),
 ('st', 'louis'),
 ('per', 'cent'),
 ('peace', 'corps'),
 ('gross', 'income'),
 ('human', 'beings'),
 ('supreme', 'court'),
 ('real', 'estate'),
 ('wage', 'rate'),
 ('kansas', 'city')]

In [ ]:
ppmi_matrix = M1_plus.to_numpy()
k = 30
top_inds = np.c_[np.unravel_index(np.argpartition(ppmi_matrix.ravel(), -k)[-k:],ppmi_matrix.shape)]

top_ppmis = []

for inds in top_inds:
    w1 = M1_plus.index[inds[0]]
    w2 = M1_plus.columns[inds[1]]
    v = M1_plus.loc[w1, w2]
    top_ppmis.append([w1, w2, v])
sorted_top_ppmis = sorted(top_ppmis, key = lambda x: x[2])
sorted_top_ppmis

[['premier', 'khrushchev', 13.065731061700156],
 ['wave', 'lengths', 13.069544114811356],
 ['racial', 'discrimination', 13.101252974538694],
 ['linear', 'operator', 13.109640760031413],
 ['pulmonary', 'vein', 13.132961834266032],
 ['anode', 'holder', 13.172996496068285],
 ['input', 'axis', 13.184242339209725],
 ['transition', 'probabilities', 13.209777431316862],
 ['sidewalk', 'cafe', 13.232497507816946],
 ['marshal', 'slim', 13.246303307341977],
 ['southeast', 'asia', 13.264918985509324],
 ['notte', 'jr', 13.302886835708344],
 ['electronic', 'switches', 13.32876546753395],
 ['bronchial', 'artery', 13.472090669907002],
 ['rob', 'roy', 13.489362146410533],
 ['urethane', 'foam', 13.496281560350324],
 ['et', 'al', 13.509337713175771],
 ['bobby', 'joe', 13.548711951509915],
 ['virgin', 'islands', 13.563703416292318],
 ['billy', 'tilghman', 13.602447117567252],
 ['pulmonary', 'artery', 13.668293567262587],
 ['san', 'francisco', 13.745310222782267],
 ['du', 'pont', 13.75517573028376],
 ['ure

# Analogies

In [ ]:
! (sed -e '1,2d' < word-test.txt) > no_header.txt
! (sed 's/\t//g' < no_header.txt) > analogies.txt

In [ ]:
analogy_sets = pd.read_csv('analogies.txt', sep=' ', header=None, names=["A1", "A2", "A3", "A4"])

In [ ]:
f = open("word-test.txt", "r")

analogy_tests = {   "semantic": {},
                    "syntactic": {}}

semantic_tests = {}
syntactic_tests = {}



for x in f:
    if "Google" in x:
        continue
    if ":" in x:
        curr_test_family = x[2:-1]
        if 'gram' in x:
            analogy_tests["syntactic"].update({curr_test_family: []})
            curr_test_type = "syntactic"
        else:
            analogy_tests["semantic"].update({curr_test_family: []})
            curr_test_type = "semantic"

    else:
        A1, A2, A3, A4 = x.split()
        if A1.lower() in W and A2.lower() in W and A3.lower() in W and A4.lower() in W:
            analogy_tests[curr_test_type][curr_test_family].append({
                "A1": A1,
                "A2": A2,
                "A3": A3,
                "A4": A4,
            })

f.close()

In [ ]:
for x in analogy_tests["semantic"].copy():
    if len(analogy_tests["semantic"][x]) == 0:
        del analogy_tests["semantic"][x]

for x in analogy_tests["syntactic"].copy():
    if len(analogy_tests["syntactic"][x]) == 0:
        del analogy_tests["syntactic"][x]

In [ ]:
sum_dict = {}
for x, test_type_dict in analogy_tests.items():
    sum_dict[x] = 0
    for i, test_type in test_type_dict.items():
        sum_dict[x] += len(test_type)

print(sum_dict)

{'semantic': 162, 'syntactic': 2045}


In [ ]:
analogies_df = pd.DataFrame(columns = ["Superfamily", "Family", "A1", "A2", "A3", "A4"])

for test_superfamily in analogy_tests:
    for test_family in analogy_tests[test_superfamily]:
        for test in analogy_tests[test_superfamily][test_family]:
            analogies_df.loc[len(analogies_df)] = [test_superfamily,
                                 test_family,
                                 test["A1"],
                                 test["A2"],
                                 test["A3"],
                                 test["A4"]]

analogies_df.head()

,Superfamily,Family,A1,A2,A3,A4
0,semantic,capital-common-countries,Berlin,Germany,London,England
1,semantic,capital-common-countries,Berlin,Germany,Moscow,Russia
2,semantic,capital-common-countries,Berlin,Germany,Paris,France
3,semantic,capital-common-countries,Berlin,Germany,Rome,Italy
4,semantic,capital-common-countries,London,England,Moscow,Russia


In [ ]:
analogies_df.to_csv("valid_analogies.csv")

In [ ]:
def complete_analogy_top_n(matrix, a1, a2, a3, n=10):
    result_vector = matrix.loc[a2] - matrix.loc[a1] + matrix.loc[a3]
    result_corrs = matrix.corrwith(result_vector, axis=1, method = cosine_sim)

    # Remove self-ref
    result_corrs.loc[a1] = -2
    result_corrs.loc[a2] = -2
    result_corrs.loc[a3] = -2

    candidates = []

    for i in range(n):
        curr_max = np.argmax(result_corrs)
        candidate = matrix.index[curr_max]
        candidates.append(candidate)
        result_corrs[curr_max] = -2

    return candidates

In [ ]:
SVD_300_Correct_Num = {}

for test_superfamily in analogy_tests:
    SVD_300_Correct_Num[test_superfamily] = 0

    for test_family in analogy_tests[test_superfamily]:
        print(test_family)
        for test in analogy_tests[test_superfamily][test_family]:
            r = complete_analogy_top_n(M2_dict[300],
                                       test["A4"].lower(), test["A4"].lower(), test["A4"].lower(),
                                       1)
            if test["A4"].lower() in r:
                SVD_300_Correct_Num[test_superfamily] += 1



print(SVD_300_Correct_Num)

capital-common-countries
capital-world
city-in-state
family
gram1-adjective-to-adverb
gram2-opposite
gram3-comparative
gram4-superlative
gram5-present-participle
gram6-nationality-adjective
gram7-past-tense
gram8-plural
gram9-plural-verbs
{'semantic': 0, 'syntactic': 0}
